In [1]:
# Загрузка библиотек
import pandas as pd
import sqlite3 as db

import datetime as dt
import multiprocessing
from multiprocessing import Pool, Manager

from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
import pprint
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from package.select_run_test import SampleComparisonTest

# Загрузим функции поиска во всех записях о товаре наиболее часто встречающихся значений признака "цвет"
# для наименований товаров, по которым есть записи с незаполненым признаком "цвет"
# (функции написаны в отдельном модуле для обеспечения требований блиотеки  multiprocessing)
from package.preparation import add_freq_product_color, freq_product_color

import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.ticker import FormatStrFormatter

from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import silhouette_score

<h1>СОДЕРЖАНИЕ РАЗДЕЛА 5<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#5.-Построение-модели-склонности-к-покупкам" data-toc-modified-id="5.-Построение-модели-склонности-к-покупкам-1">5. Построение модели склонности к покупкам</a></span><ul class="toc-item"><li><span><a href="#5.1.-Загрузка-датасета,-полученного-в-разделе-3" data-toc-modified-id="5.1.-Загрузка-датасета,-полученного-в-разделе-3-1.1">5.1. Загрузка датасета, полученного в разделе 3</a></span></li><li><span><a href="#5.2.-Формирование-датасета-покупателей,-склонных-к-покупке" data-toc-modified-id="5.2.-Формирование-датасета-покупателей,-склонных-к-покупке-1.2">5.2. Формирование датасета покупателей, склонных к покупке</a></span></li><li><span><a href="#5.3.-Проверка-склонность-к-покупкам-покупателей-страны-32-города-1188" data-toc-modified-id="5.3.-Проверка-склонность-к-покупкам-покупателей-страны-32-города-1188-1.3">5.3. Проверка склонность к покупкам покупателей страны 32 города 1188</a></span></li></ul></li></ul></div>

# 5. Построение модели склонности к покупкам

## 5.1. Загрузка датасета, полученного в разделе 3

In [2]:
# Загрузим итоговый датаcет из хранилища
connection = db.connect('data/purchases.db')
print("База данных подключена")
query = 'SELECT * FROM full_table'
df_all = pd.read_sql_query(query, connection)
print("Таблица загружена")
connection.close()
print("Соединение с базой данных закрыто")

База данных подключена
Таблица загружена
Соединение с базой данных закрыто


## 5.2. Формирование датасета покупателей, склонных к покупке

**Определим характеристику "склонность клиента к покупке":**  
Склонность клиента к покупке это признак того что после коммуникации с клиентом в успешной рекламной кампании клиент купил больше одного товара.

In [3]:
# Сформируем датасет клиентов, участвовавших в первой (успешной) маркетинговой кампании и получивших скидку
df_client = df_all[((df_all.test == 1) & (df_all.dt<16)) | ((df_all.city == 1134) & \
                                                            ((df_all.dt == 14) | (df_all.dt == 44)))]
df_client.reset_index(drop=False, inplace=True)

In [4]:
# Функция разметки клиентов на тех кто совершил одну покупку и несколько покупок
def any_pusher(df, x):
    if len(df[df['id']==x]) > 1: return 1
    else: return 0

In [5]:
# Проставим прзнак нескольких покупок
df_client['inclination'] = df_client['id'].apply(lambda x: any_pusher(df_client, x))

/var/folders/5h/1vkpjwr90l19cmzfgsfrcpvm0000gp/T/ipykernel_25883/1335346606.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_client['inclination'] = df_client['id'].apply(lambda x: any_pusher(df_client, x))


In [6]:
# Проверим сбалансированность количества клиентов в датасете
df = df_client.groupby('id').agg({'gender': 'max', 'age': 'max', 'education': 'max',
                                  'city': 'max', 'country': 'max',
                                  'cost': 'sum', 'product': 'count', 'colour': 'count', 'product_sex': 'count',
                                  'personal_coef': 'max', 'inclination': 'max'})
len(df[df['inclination']==1]), len(df[df['inclination']==0])

(6313, 3397)

Перед обучением модели целесообразно проверить данные на аномалии и исключить такие данные из обучающего датасета.  
1) При проверки датасета на аномальные записи используем три метода, результаты которых дополним друг другом для повышения качества обучающей выборки.  
2) При проверки данных на аномалии не будем учитывать значения признаков `id`, `lbt_coef`, `ac_coef`, `sm_coef` - так как эти данные уникальны почти для каждого покупателя, а значит не принесут пользы для факторной оценки данных и дальнейшего обучения модели.  
3) При использовании методов попризнакового выявления аномалий не будем учитывать признаки `product_sex`,`base_sale`,`education`, `gender`, `test` - так как они имеют понятные уникальные значения без аномалий (выявлено визуально). Также не будем учитывать признак `inclination` - так как это целевая переменная для обуения модели.

In [7]:
# Определим колонки для попризнакового выявления аномалий
check_columns = ['product', 'colour', 'cost', 'product_sex', 'base_sale', 'dt',
                 'gender', 'age', 'education', 'city', 'country', 'personal_coef']

In [8]:
# Проведём кодирование каждого признака 
df_fit = pd.DataFrame()
for column in check_columns:
    le = LabelEncoder()
    df_fit[column] = le.fit_transform(df_client[column])

In [9]:
# Инициализируем словарь индексов, в котором будем фиксировать:
# метод выявления аномалий; список индексов записей-аномалий
indexs = {'Quantile': []}

In [10]:
# Заполним словарь индексов записей-аномалий, индексами аномалий,
# выявленными квантилями более 3-сигма
for column in df_fit.columns:
    quantile_indexs = list(df_fit[df_fit[column] > df_fit[column].quantile(0.997)].index)
    if len(quantile_indexs) > 0:
        indexs['Quantile'].extend(quantile_indexs)
indexs['Quantile'] = list(set(indexs['Quantile']))

In [11]:
# Заполним словарь индексов записей-аномалий, индексами аномалий,
# выявленными Локалфактором и Изолесом
lof = LocalOutlierFactor(n_neighbors=693, n_jobs=-1)
ifo = IsolationForest(max_samples=12300, n_jobs=-1, random_state=42)
pred_lof = lof.fit_predict(df_fit)
pred_ifo = ifo.fit_predict(df_fit)
indexs['LocalOutlierFactor'] = list(df_fit[pred_lof == -1].index)
indexs['IsolationForest'] = list(df_fit[pred_ifo == -1].index)

In [12]:
# Сформируем единый список индексов, полученных всеми методами
indexs_all = []
# Определим долю выбросов, выявленную каждым методом
for method in indexs.keys():
    count_anomaly = len(indexs[method])
    print(f"Методом '{method}' выявлено {count_anomaly} записей-аномалий,", end=' ')
    print(f"что составляет {round(count_anomaly/len(df_client) * 100, 2)}% всех записей")
    indexs_all.extend(indexs[method])
indexs_all = list(set(indexs_all))
indexs_all.sort()
print(f'\nИТОГО вывленных записей-аномалий {len(indexs_all)},', end=' ')
print(f'что составляет {round(len(indexs_all)/len(df_client) * 100, 2)}% всех записей')

Методом 'Quantile' выявлено 507 записей-аномалий, что составляет 1.31% всех записей
Методом 'LocalOutlierFactor' выявлено 400 записей-аномалий, что составляет 1.04% всех записей
Методом 'IsolationForest' выявлено 7864 записей-аномалий, что составляет 20.38% всех записей

ИТОГО вывленных записей-аномалий 8207, что составляет 21.27% всех записей


In [14]:
# Удалим из обучающего датасета записи с признаками аномалий
df_client = df_client.drop(index=indexs_all)

In [15]:
# Проверим сбалансированность количества клиентов в датасете
df = df_client.groupby('id').agg({'gender': 'max', 'age': 'max', 'education': 'max',
                                  'city': 'max', 'country': 'max',
                                  'cost': 'sum', 'product': 'count', 'colour': 'count', 'product_sex': 'count',
                                  'personal_coef': 'max', 'inclination': 'max'})
# Проверим сбалансированность записей датасета по целевому признаку
balance = df.inclination.value_counts()
print(f'Положительный класс: {balance[1]} ({round(balance[1]/(balance[0]+balance[1])*100)})%')
print(f'Отрицательный класс: {balance[0]} ({round(balance[0]/(balance[0]+balance[1])*100)})%')

Положительный класс: 5617 (66)%
Отрицательный класс: 2853 (34)%


**Вывод:** Так как датасет неполностью сбалансирован, снизим разбалансировку до соотношения 57%/43% и используем при обучении моделей параметр `class_weight='balanced'`, который позволяет учесть разбалансировку такого масштаба. Для этого достаточно исключитить из положительного класса 6пп, то есть около 1800 записей.

In [16]:
# Соберём итоговый датасет
df_train_test = pd.concat([df[df.inclination==1][:3840], df[df.inclination==0]])

In [17]:
# Проверим сбалансированность записей датасета по целевому признаку
balance_1 = df_train_test.inclination.value_counts()
print(f'Положительный класс: {balance_1[1]} ({round(balance_1[1]/(balance_1[0]+balance_1[1])*100)})%')
print(f'Отрицательный класс: {balance_1[0]} ({round(balance_1[0]/(balance_1[0]+balance_1[1])*100)})%')

Положительный класс: 3840 (57)%
Отрицательный класс: 2853 (43)%


In [18]:
# Закодируем категориальный признак 'education':
le = LabelEncoder()
le.fit(list(set(df_train_test['education'].dropna().unique())))
df_train_test['education'] = le.transform(df_train_test['education'])
df_train_test.head(3)

,gender,age,education,city,country,cost,product,colour,product_sex,personal_coef,inclination
id,,,,,,,,,,,
4,0.0,35,1,1134,32,23095,5,5,5,0.5072,1
6,1.0,52,1,1188,32,27013,7,7,7,0.4304,1
18,1.0,53,1,1188,32,27535,5,5,5,0.4304,1


In [19]:
# Стандартизуем не бинарные признаки, которые будем использовать при обучении
ss = StandardScaler()
for col in ['age', 'city', 'country', 'cost', 'product', 'colour', 'product_sex', 'personal_coef']:
    df_train_test[col] = ss.fit_transform(df_train_test[[col]])
df_train_test.head(3)

,gender,age,education,city,country,cost,product,colour,product_sex,personal_coef,inclination
id,,,,,,,,,,,
4,0.0,-0.514639,1,-0.122852,0.0,0.366064,0.47064,0.47064,0.47064,1.043658,1
6,1.0,1.104678,1,0.627286,0.0,0.542331,0.99027,0.99027,0.99027,-0.859862,1
18,1.0,1.199932,1,0.627286,0.0,0.565815,0.47064,0.47064,0.47064,-0.859862,1


In [20]:
# Признаки используемые при обучении модели
cols = ['gender', 'age', 'education', 'city', 'country', 'cost', 'product',
       'colour', 'product_sex', 'personal_coef']

In [21]:
# Подготовим датасет для обучения
X_train, X_test, y_train, y_test = train_test_split(df_train_test[cols],
                                                    df_train_test['inclination'],
                                                    test_size=0.3, random_state=42)

In [22]:
# Обучим модель "Линейная регресия"
lr = LogisticRegression(class_weight='balanced')
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(f'Метрика: {accuracy_score(y_test, y_pred)}')

Метрика: 0.9591633466135459


In [23]:
# Обучим модель "Случайный лес Деревьев принятия решения"
rf = RandomForestClassifier(class_weight='balanced')
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(f'Метрика: {accuracy_score(y_pred, y_test)}')

Метрика: 0.9646414342629482


In [24]:
# Обучим модель "Нейронная сеть - ПЕРЦЕПТРОН"
mlp = MLPClassifier()
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)
print(f'Метрика: {accuracy_score(y_pred, y_test)}')

Метрика: 0.9641434262948207


**Вывод:** Метрики трёх алгоритмов машинного обучения показали хорошее качество. В качестве модели для прогннозирования признака `inclination` выбирем алгоритм нейросети "Пепцептрон".

In [25]:
# Обучим модель "Случайный лес Деревьев принятия решения" на всех имеющихся данных 
mlp = MLPClassifier()
mlp.fit(df_train_test[cols], df_train_test['inclination'])

MLPClassifier()

## 5.3. Проверка склонность к покупкам покупателей страны 32 города 1188

In [26]:
# Определим покупателей страны 32 города 1188
df_32_1188 = df_all[(df_all.country==32) & (df_all.city==1188)]
df_32_1188_client = df_32_1188.groupby('id').agg({'gender': 'max', 'age': 'max', 'education': 'max',
                                  'city': 'max', 'country': 'max',
                                  'cost': 'sum', 'product': 'count', 'colour': 'count', 'product_sex': 'count',
                                  'personal_coef': 'max'})

In [27]:
# Закодируем категориальный признак 'education':
le = LabelEncoder()
le.fit(list(set(df_32_1188_client['education'].dropna().unique())))
df_32_1188_client['education'] = le.transform(df_32_1188_client['education'])
# Стандартизуем не бинарные признаки, которые будем использовать при обучении
ss = StandardScaler()
for col in ['age', 'city', 'country', 'cost', 'product', 'colour', 'product_sex', 'personal_coef']:
    df_32_1188_client[col] = ss.fit_transform(df_32_1188_client[[col]])

In [28]:
# Определим сколько покупателей страны 32 города 1188 склонны к покупкам
df_32_1188_client['inclination'] = mlp.predict(df_32_1188_client)

In [29]:
# Посчитаем баланс клиентов склонных и не склонных к покупкам в стране 32 городе 1188
balance_2 = df_32_1188_client.inclination.value_counts()
print(f'Положительный класс: {balance_2[1]} ({round(balance_2[1]/(balance_2[0]+balance_2[1])*100)})%')
print(f'Отрицательный класс: {balance_2[0]} ({round(balance_2[0]/(balance_2[0]+balance_2[1])*100)})%')

Положительный класс: 9657 (78)%
Отрицательный класс: 2781 (22)%


In [30]:
# Посчитаем баланс клиентов склонных и не склонных к покупкам в городе с проведённой рекламной кампанией
balance_2 = df.inclination.value_counts()
print(f'Положительный класс: {balance_2[1]} ({round(balance_2[1]/(balance_2[0]+balance_2[1])*100)})%')
print(f'Отрицательный класс: {balance_2[0]} ({round(balance_2[0]/(balance_2[0]+balance_2[1])*100)})%')

Положительный класс: 5617 (66)%
Отрицательный класс: 2853 (34)%


**Вывод:** В городе 1188 страны 32 клиентов сколнных к покупке на 12пп больше чем в горде, в котором была проведена рекламная кампания, принёсшая значительный прирост выручки и оптимизацию частоты покупок клиентами. Таким образом, проводить подобную рекламную кампанию в городе 1188 страны 32 целесообразно, в том числе с учётом дополнительных её доработок, рекомендованных в п.3 и п.4 исслдедования.